Name: Rahul Anand

SRN: PES2UG23CS461

Part 3a 


In [1]:
%pip install python-dotenv --upgrade --quiet langchain langchain-groq

from dotenv import load_dotenv
load_dotenv()

from langchain_groq import ChatGroq

llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0.0)

Note: you may need to restart the kernel to use updated packages.


In [2]:
question = "Sarah has 3 guitars. She buys 2 more cases of guitar strings. Each case has 6 packs of strings. How many packs of strings does she have now?"

prompt_standard = f"Answer this question: {question}"
print("--- Standard ---")
print(llm.invoke(prompt_standard).content)

--- Standard ---
To find out how many packs of strings Sarah has now, we need to calculate the total number of packs she has. 

She initially has 3 guitars, but the number of guitars doesn't directly relate to the number of packs of strings. 

She buys 2 more cases of guitar strings, and each case has 6 packs of strings. 

So, the total number of packs of strings she bought is 2 cases * 6 packs/case = 12 packs.

Since she already had some packs of strings (but we don't know how many), we'll call the initial number of packs 'x'. 

The total number of packs of strings she has now is x (initial packs) + 12 (new packs).

However, the problem doesn't provide information about the initial number of packs of strings. If we assume she had 0 packs initially, then the total number of packs of strings she has now is 12.


In [3]:
prompt_cot = f"Answer this question. Let's think step by step. {question}"

print("--- Chain of Thought ---")
print(llm.invoke(prompt_cot).content)

--- Chain of Thought ---
To find out how many packs of strings Sarah has now, we need to follow these steps:

1. First, let's find out how many packs of strings are in the 2 cases she bought. Since each case has 6 packs of strings, we multiply the number of cases by the number of packs per case:
   2 cases * 6 packs/case = 12 packs

2. Now, we need to add the packs of strings she already had to the packs she just bought. We know she had 3 guitars, but we don't know how many packs of strings she had for each guitar. However, since we're not given any information about the number of strings per guitar, we'll assume she had a standard set of strings for each guitar, which is usually 6 strings per guitar. So, she had 3 guitars * 6 strings/guitar = 18 strings. But since we're looking for packs of strings, and each pack contains 6 strings, we divide 18 by 6 to get the number of packs she had:
   18 strings / 6 strings/pack = 3 packs

3. Now, we add the packs she bought to the packs she alrea

Part 3b

In [4]:
# Setup
%pip install python-dotenv --upgrade --quiet langchain langchain-groq

from dotenv import load_dotenv
load_dotenv()

from langchain_groq import ChatGroq

llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0.7) 

Note: you may need to restart the kernel to use updated packages.


In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

problem = "How can I learn to play the piano as a beginner?"

prompt_branch = ChatPromptTemplate.from_template(
    "Problem: {problem}. Give me one unique, creative solution. Solution {id}:"
)

branches = RunnableParallel(
    sol1=prompt_branch.partial(id="1") | llm | StrOutputParser(),
    sol2=prompt_branch.partial(id="2") | llm | StrOutputParser(),
    sol3=prompt_branch.partial(id="3") | llm | StrOutputParser(),
)

prompt_judge = ChatPromptTemplate.from_template(
    """
    I have three proposed solutions for: '{problem}'
    
    1: {sol1}
    2: {sol2}
    3: {sol3}
    
    Act as a Music Teacher. Pick the most effective one for a beginner and explain why.
    """
)

tot_chain = (
    RunnableParallel(problem=RunnableLambda(lambda x: x), branches=branches)
    | (lambda x: {**x["branches"], "problem": x["problem"]}) 
    | prompt_judge
    | llm
    | StrOutputParser()
)

print("--- Tree of Thoughts (ToT) ---")
print(tot_chain.invoke(problem))

--- Tree of Thoughts (ToT) ---
As a music teacher, I highly recommend **Solution 2: The "Piano Play-Along" with YouTube Channels** as the most effective approach for a beginner. Here's why:

1. **Accessibility**: YouTube is an incredibly accessible platform, and there are countless piano play-along channels available. Beginners can easily find videos that suit their skill level and musical interests.
2. **Guidance**: Play-along videos offer step-by-step guidance, allowing beginners to learn at their own pace. This is particularly helpful for those who may feel overwhelmed by traditional lessons or online courses.
3. **Variety**: Piano play-along channels cater to diverse musical styles, from classical to pop, jazz, and rock. This variety keeps learning engaging and exciting, preventing boredom and plateaus.
4. **Real-time feedback**: By playing along with a video, beginners receive instant feedback on their playing. This helps them identify areas for improvement and makes the learning 

In [6]:
prompt_draft = ChatPromptTemplate.from_template(
    "Write a 1-sentence song idea about: {topic}. Genre: {genre}."
)

drafts = RunnableParallel(
    draft_scifi=prompt_draft.partial(genre="Rock") | llm | StrOutputParser(),
    draft_romance=prompt_draft.partial(genre="Jazz") | llm | StrOutputParser(),
    draft_horror=prompt_draft.partial(genre="Classical") | llm | StrOutputParser(),
)

prompt_combine = ChatPromptTemplate.from_template(
    """
    I have three song ideas for the topic '{topic}':
    1. Rock: {draft_scifi}
    2. Jazz: {draft_romance}
    3. Classical: {draft_horror}
    
    Your task: Create a new Fusion Song that combines the ENERGY of Rock, the IMPROVISATION of Jazz, and the ELEGANCE of Classical.
    Write one paragraph.
    """
)

got_chain = (
    RunnableParallel(topic=RunnableLambda(lambda x: x), drafts=drafts)
    | (lambda x: {**x["drafts"], "topic": x["topic"]}) 
    | prompt_combine
    | llm
    | StrOutputParser()
)

print("--- Graph of Thoughts (GoT) ---")
print(got_chain.invoke("Summer Nights"))

--- Graph of Thoughts (GoT) ---
"Midnight Oasis" is a fusion song that embodies the essence of a summer night, combining the unbridled energy of rock, the improvisational freedom of jazz, and the refined elegance of classical music. The song begins with a driving rock beat, propelled by a pulsing guitar riff, before seamlessly transitioning into a sultry jazz solo, courtesy of a soaring saxophone melody, evoking the city's vibrant nightlife. As the night wears on, the music evolves into a majestic classical nocturne, with a grand piano accompaniment, transporting listeners to a moonlit garden, where fireflies dance in time with the music, their twinkling lights mirroring the stars above. Throughout the song, the boundaries between genres blur, creating a unique and captivating soundscape that captures the magic of a summer night, where anything is possible under the radiant glow of the midnight sky.
